# Simple model for [Kaggle kernel](https://www.kaggle.com/artgor/quick-and-dirty-regressio)


In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import json
pd.set_option('display.max_columns', 1000)

In [2]:
import os,psutil

process = psutil.Process(os.getpid())
print('Used Memory:',process.memory_info().rss / 1024 / 1024,'MB')

Used Memory: 128.43359375 MB


In [3]:
from utils import kaggle_util

# read data
train, test, train_labels, specs, sample_submission = kaggle_util.read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code,\
activities_labels, assess_titles, list_of_event_id, all_title_event_code = kaggle_util.encode_title(train, test, train_labels)
# tranform function to get the train and test set
reduce_train, reduce_test, categoricals = kaggle_util.get_train_and_test(train, test, win_code, list_of_user_activities, list_of_event_code,\
activities_labels, assess_titles, list_of_event_id, all_title_event_code)


Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns


1
0
0
11
1
0
0
0
1
1
1
0
1
0
0
0
0
4
1
1
0
0
1
0
1
1
1
0
0
5
1
0
0
0
0
1
0
0
0
2
0
11
0
0
0
5
1
1
1
10
1
0
0
7
1
0
1
0
1
1
1
4
0
0
1
0
1
1
1
0
1
0
0
4
1
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
1
1
0
0
1
0
0
0
0
0
8
1
2
0
7
0
4
1
0
1
2
0
6
0
3
0
0
0
7
0
6
0
2
0
1
0
3
1
0
1
2
0
0
0
0
0
0
1
1
0
0
1
0
0
0
1
0
1
0
1
0
1
1
1
0
1
0
1
1
1
0
0
0
1
0
1
1
1
0
0
5
1
2
1
0
1
0
1
0
0
0
0
1
1
0
1
2
1
0
1
1
1
0
0
0
1
0
1
2
1
0
1
0
1
10
1
0
1
0
0
2
0
2
0
0
0
0
0
6
1
0
0
4
0
2
1
0
0
3
1
0
1
0
0
7
0
1
0
3
1
1
0
4
1
0
0
2
1
0
1
2
1
3
1
0
0
0
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
4
1
0
1
0
1
0
0
0
1
0
1
0
1
0
1
0
0
1
1
0
1
0
1
0
0
4
0
0
0
0
1
1
1
1
1
0
1
1
1
0
1
0
0
2
1
0
0
0
0
3
0
1
0
1
0
6
0
1
1
2
0
3
1
0
1
2
1
0
1
0
0
3
1
1
1
0
0
1
0
2
1
1
1
0
1
0
1
3
1
0
1
0
0
3
1
0
0
0
1
0
1
0
1
0
1
0
1
0
0
7
1
1
0
9
0
0
1
0
1
0
1
0
1
1
1
3
1
0
1
1
1
0
0
3
1
1
1
6
0
0
0
1
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
0
0
1
4
1
1
1
0
0
1
1
0
1
0
1
1
1
0
0
0
1
0
1
1


0
0
0
2
0
0
0
0
1
0
1
0
1
0
1
0
1
0
0
0
1
1
0
0
1
0
0
2
1
0
1
0
1
0
1
0
1
0
1
0
0
0
1
0
0
3
1
3
1
2
1
0
1
0
1
1
1
0
1
1
1
0
1
0
1
2
1
0
1
2
1
0
0
1
0
2
0
0
0
1
0
1
1
0
1
3
0
0
1
0
0
3
1
0
0
4
1
0
1
0
1
0
1
0
0
1
0
0
1
0
0
3
1
0
1
2
0
0
1
0
0
0
1
1
1
0
0
8
1
0
1
1
0
0
1
0
1
1
1
0
1
0
1
0
0
0
1
0
0
1
1
0
0
2
0
5
1
0
1
0
1
0
1
0
1
3
0
9
0
0
0
0
0
0
0
3
1
1
0
1
0
6
1
0
1
0
0
6
0
2
1
0
1
0
1
0
1
0
1
0
1
1
1
0
1
0
1
1
0
0
1
0
1
0
1
0
1
0
0
3
0
1
1
0
1
0
1
0
1
1
1
0
1
0
1
3
1
0
0
0
0
2
0
0
1
0
1
0
1
0
1
0
1
1
0
0
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
0
1
0
1
0
1
0
1
1
0
4
1
0
1
2
1
0
1
0
0
0
0
0
1
0
1
0
1
5
1
0
1
0
1
0
1
1
1
1
1
0
1
0
1
0
1
2
0
0
1
0
1
0
1
0
1
1
1
0
0
3
0
0
1
1
1
0
0
2
0
0
1
0
1
8
1
3
1
3
1
0
1
3
1
0
1
1
0
0
1
0
1
3
1
0
1
0
0
1
0
0
1
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
1
2
1
0
1
0
1
1
1
0
1
6
0
1
1
0
0
0
0
3
1
2
0
0
1
3
1
1
1
0
1
0
1
0
0
1
1
1
1
1
1
1
1
0
1
0
0
3
0
2
1
0
1
0
1
0
1
0
0
2
1
0
1
3
1
0
1
1
1
1
0
21
1
0
1
3
1
1
1
0
1
2
1
2
0
4
1
0
1
0
1
4
1
0
1
0
1
0
1
2
1
0
1
9
1
1
1
3
1
1
1
0

1
0
1
0
1
0
1
2
0
1
1
0
1
0
1
1
1
1
1
0
1
0
1
1
0
8
1
0
1
2
1
0
1
0
1
0
1
0
1
1
0
6
1
1
0
3
0
4
1
0
0
2
0
3
0
1
0
2
0
3
0
0
0
5
1
2
1
1
1
0
1
1
0
8
1
0
1
0
0
3
0
5
0
1
1
2
1
0
0
4
1
1
0
1
0
4
0
0
0
0
0
7
1
2
0
1
1
0
1
3
0
5
0
4
0
12
1
0
0
1
0
0
1
0
1
0
0
2
1
0
1
2
1
0
1
0
1
0
0
7
1
2
1
1
0
1
1
1
0
3
0
2
1
5
0
0
1
0
1
1
1
1
1
8
0
8
1
0
0
0
1
2
0
1
0
3
0
3
1
0
1
0
1
0
1
15
1
1
1
0
1
0
1
1
1
0
1
0
1
0
1
0
0
4
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
0
0
4
1
39
0
0
1
0
0
4
1
1
1
0
1
0
0
4
1
1
1
0
1
4
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
0
1
0
1
2
0
0
1
0
1
3
1
4
1
3
1
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
1
1
1
14
1
0
0
2
1
0
1
1
0
0
0
0
1
1
1
0
1
2
1
0
1
4
1
3
1
0
1
1
1
0
1
0
1
0
1
4
1
0
0
0
1
8
0
0
1
0
1
0
1
3
0
3
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
6
1
13
1
0
1
1
1
0
1
0
1
1
1
1
1
0
0
0
1
0
1
17
1
0
1
0
1
0
1
1
0
0
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
2
1
0
1
0
1
3
1
0
1
1
1
1
1
0
1
0
1
0
1
0
0
1
1
3
0
2
0
0
1
0
0
3
0
4
0
0
1
1
1
1
1
0
1
0
1
0
1
0
1
0
1
2
1
1
1
5
1
1
1
1
0
1
1
2
0
1
1


0
2
0
4
0
1
1
0
0
6
1
3
0
1
1
0
1
0
0
3
1
1
1
0
1
0
1
0
1
0
1
2
0
0
0
0
0
6
0
12
0
0
0
0
0
0
0
7
0
1
0
0
0
0
0
4
0
1
0
3
0
1
1
0
0
0
1
0
0
0
1
0
1
0
1
0
0
0
1
0
0
3
0
0
0
0
1
0
1
0
0
0
1
0
0
3
1
0
1
0
1
3
1
5
1
0
0
4
1
0
1
5
0
5
0
0
1
0
1
0
1
1
1
0
1
0
1
1
1
0
1
0
1
0
1
0
0
0
0
2
0
2
0
0
0
0
1
0
1
0
1
0
1
8
1
0
0
5
1
2
0
6
1
1
0
13
1
0
1
0
1
0
1
1
1
0
0
0
1
1
1
0
1
0
1
0
1
4
1
0
0
3
1
6
1
0
1
0
1
0
1
0
1
0
1
0
1
4
1
2
0
0
0
0
1
0
0
0
1
0
0
0
1
0
1
0
1
0
1
0
1
0
1
5
1
0
1
0
1
1
0
3
1
0
1
0
1
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
0
2
1
1
1
5
0
0
1
0
1
2
0
0
1
5
0
11
1
1
1
1
1
1
1
1
1
3
0
2
1
0
0
0
1
2
0
3
0
0
1
0
1
8
1
6
1
0
1
0
1
1
0
4
1
1
1
0
1
0
1
0
0
1
1
0
1
1
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
0
1
1
1
0
0
5
0
0
1
0
1
0
1
0
1
2
1
1
1
0
1
0
1
2
1
0
1
0
1
0
1
0
0
0
1
0
0
5
1
1
1
1
1
5
1
0
1
0
1
1
0
4
1
0
1
0
1
0
1
0
0
0
0
0
0
1
0
1
0
0
0
2
1
0
1
4
0
0
1
0
1
1
1
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
1
0
0
4
1
1
1
2
1
9
1
1
1
1
1
8
0
0
0
0
0
0
0
3
0
3
1
0
1
0
1
2
1
1
1
0
1
0
0
4
1
0
0
1
0
3
0
0
1
0
1

...

1
0
0
3
0
6
0
0
0
2
0
1
0
0
1
0
0
0
1
2
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
1
1
2
0
3
1
0
1
0
1
0
1
0
1
0
1
4
1
0
0
0
1
0
1
0
1
0
1
0
1
0
1
1
1
0
1
7
0
0
0
0
1
0
1
2
1
1
1
0
1
1
1
0
1
8
1
0
1
5
1
20
1
1
1
2
0
0
1
0
1
1
0
1
1
0
1
0
0
0
1
0
1
4
1
2
0
0
1
0
0
4
1
0
1
1
0
0
1
0
1
0
1
0
0
0
0
0
1
0
0
1
1
0
1
0
1
0
1
0
1
0
0
0
0
6
0
1
0
5
0
2
0
0
0
0
1
0
1
0
1
0
0
1
0
1
1
0
1
0
1
0
1
0
0
1
0
0
0
0
0
7
0
0
0
1
0
0
0
0
0
0
1
2
1
0
0
4
1
0
0
0
0
3
1
1
0
0
0
6
1
0
0
1
0
4
0
0
1
0
1
0
0
5
1
0
1
3
1
0
1
0
0
1
1
0
1
3
0
2
1
0
0
0
0
0
0
0
1
0
0
1
1
0
1
3
1
6
1
0
1
1
0
5
1
0
1
2
1
14
0
1
1
0
1
0
1
1
1
0
1
9
1
0
1
1
1
0
1
17
1
0
1
0
1
0
1
1
1
0
1
1
1
0
1
0
1
0
0
0
1
0
0
0
0
5
0
0
0
0
1
0
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
4
0
2
0
5
0
4
0
1
1
0
0
4
1
1
1
0
0
0
1
1
0
0
1
2
1
8
0
7
1
0
1
0
1
0
1
0
0
0
0
0
0
0
0
6
1
0
1
0
0
0
1
2
1
0
1
0
0
3
1
0
0
1
1
5
1
1
0
1
1
0
1
0
0
8
0
1
0
0
1
2
1
0
1
0
1
1
0
0
1
0
1
0
1
0
1
0
1
0
0
3
0
2
1
0
1
0
1
1
1
0
1
3
1
17
0
1
0
4
1
0
0
0
1
0
1
0
0
0
1
0
1
0
1
0
1
2
1
0
1
1
1
1
0
2


1
1
1
0
1
3
1
0
1
0
1
1
0
2
0
1
1
0
0
7
0
0
1
15
0
0
0
14
0
5
1
3
0
5
0
0
1
2
1
1
1
0
1
0
1
1
0
0
1
0
0
0
1
3
1
1
1
0
1
5
1
1
0
0
1
0
0
0
1
1
0
1
1
0
1
0
1
0
1
0
1
2
1
5
1
0
0
0
0
0
1
0
1
2
1
0
1
1
1
0
1
0
1
0
0
2
0
1
1
0
0
0
1
1
1
0
0
0
1
4
1
0
1
0
1
5
0
1
0
2
1
0
0
2
0
4
1
13
0
8
1
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
1
1
0
1
1
5
1
2
1
0
1
0
0
3
1
0
1
1
1
0
1
0
1
0
1
0
0
0
0
0
1
0
1
0
0
0
1
1
1
2
0
0
1
2
0
1
1
0
1
0
1
0
1
0
0
0
1
1
1
2
0
0
0
0
1
1
1
1
0
1
1
0
1
0
1
0
0
0
1
0
1
0
0
8
0
1
0
3
1
0
1
2
1
0
0
1
0
1
0
2
1
0
1
6
1
0
1
0
1
0
0
0
0
0
1
0
1
0
1
0
1
1
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
0
1
0
1
1
1
0
1
0
1
0
1
0
1
0
1
19
1
0
1
0
0
3
1
1
1
11
1
1
0
4
0
3
0
0
1
5
1
0
1
0
1
0
1
0
1
0
1
0
0
1
0
0
0
0
1
4
1
0
1
0
1
2
0
0
1
0
1
1
0
0
0
0
0
11
0
0
0
0
1
1
0
0
0
3
0
3
0
4
0
18
0
0
0
1
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
1
0
2
0
0
0
2
1
1
1
2
0
0
1
0
0
0
1
0
1
2
1
0
1
0
1
0
0
0
1
0
1
5
0
0
1
2
1
0
1
2
0
3
0
0
1
1
0
1
0
1
1
0
0
0
0
1
0
1
1
0
1
0
1
0
1
0
0
1
1
0
1
2
0
1
0
0
0
1
0
1
1
0
0
6
1
0
1
0
1
0
1

1
0
0
1
0
0
0
0
0
0
1
0
1
0
1
1
1
8
1
3
1
2
1
3
1
0
1
7
1
1
1
0
0
0
1
3
1
0
1
1
0
2
0
0
1
0
0
0
0
3
0
0
1
0
1
0
1
0
0
3
1
0
1
0
1
0
1
3
1
0
1
0
1
0
1
0
0
2
0
0
0
0
1
1
1
0
1
0
1
6
0
0
0
4
0
0
0
1
0
0
1
0
0
2
0
3
0
1
0
0
1
0
0
1
1
0
1
0
1
0
1
1
1
1
1
1
1
0
1
0
1
0
0
1
1
0
1
0
1
0
0
3
1
1
1
4
1
1
1
0
1
0
1
2
0
7
1
0
1
0
1
0
1
2
0
1
1
0
1
0
1
2
1
0
1
2
0
2
1
0
0
4
0
4
0
6
0
0
0
3
0
2
0
6
0
3
0
2
0
1
0
2
0
0
0
0
0
0
1
1
0
0
0
0
1
0
0
1
0
3
0
1
0
0
0
1
1
7
1
0
1
1
0
0
1
2
1
7
1
17
0
8
1
0
1
0
1
0
0
14
1
0
0
1
1
0
1
1
1
2
1
0
1
0
0
0
1
3
1
2
1
0
0
7
1
0
0
0
1
0
1
1
0
0
0
3
1
1
1
0
1
0
1
0
0
1
1
0
1
0
1
0
1
1
1
2
0
0
1
0
0
2
1
0
1
1
0
0
1
0
0
11
1
0
1
0
0
2
1
0
0
0
0
11
0
6
0
12
0
2
1
1
1
0
1
0
0
0
0
0
1
2
0
0
1
0
0
1
1
2
1
2
0
0
1
2
1
0
0
8
0
15
0
1
0
0
0
5
0
5
1
1
0
5
0
1
1
0
0
3
1
0
0
5
1
0
1
0
0
0
0
0
0
1
0
0
1
0
1
0
0
2
1
0
1
3
1
0
1
2
0
1
0
1
0
9
1
1
0
11
0
1
0
1
1
0
0
0
1
0
0
4
1
0
1
0
1
1
1
0
0
5
0
5
1
0
1
0
1
5
0
4
0
1
0
7
1
0
1
0
0
0
1
0
1
0
1
4
1
0
1
1
0
3
0
0
0
2
1
0
1
0
1
1
1
0
1

1
0
0
0
1
1
1
0
0
3
1
1
1
2
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
1
0
0
0
1
0
1
0
1
2
0
0
1
0
0
0
0
2
0
11
1
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
2
1
0
1
0
0
4
0
0
1
0
0
0
0
0
1
0
0
3
1
0
1
2
1
0
0
2
1
0
1
0
1
0
1
0
0
2
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
5
0
3
0
0
1
0
1
0
1
1
1
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
9
0
2
0
1
0
2
0
2
0
0
1
3
0
8
0
0
0
1
0
2
0
2
0
1
0
1
0
3
1
0
0
0
1
1
0
0
0
0
1
2
0
0
1
0
0
1
0
0
1
0
1
1
1
1
1
0
0
0
1
1
0
4
1
0
1
7
0
0
1
5
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
2
1
1
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
0
0
0
0
0
5
0
0
1
0
0
1
0
0
1
2
0
0
1
0
1
0
1
0
1
2
0
1
0
0
0
2
1
0
0
0
1
0
1
0
0
1
0
0
1
0
0
3
1
1
0
3
0
1
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
1
0
1
1
0
0
0
6
0
0
0
0
0
2
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
1
2
0
0
0
0
0
0
1
0
1
2
1
0
1
1
0
0
1
0
0
0
0
4
0
2
0
1
0
0
0
1
0
0
1
2
0
0
0
0
0
0
0
0
0
1
1
4
0
0
0
0
0
0
1
0
1
0
1
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0

1
4
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
1
0
1
2
0
0
1
0
0
0
1
3
0
0
1
0
1
1
0
0
1
0
1
0
1
6
0
0
0
0
1
4
1
0
1
0
0
4
0
0
0
0
0
0
1
0
0
0
1
2
0
2
1
1
1
0
0
0
1
1
0
0
1
1
0
5
0
0
1
0
1
1
1
0
1
0
0
3
0
0
0
0
1
0
1
0
1
0
1
5
0
0
1
1
1
0
0
0
1
0
0
0
1
0
0
6
0
2
0
0
1
0
0
7
1
0
1
0
1
1
1
4
0
0
0
0
0
6
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
2
0
0
1
0
0
0
1
0
0
0
0
0
0
2
0
0
0
0
0
0
1
0
1
0
0
2
0
0
1
0
0
0
1
3
1
6
0
0
1
0
1
9
1
1
1
1
1
0
0
0
1
0
0
0
1
0
1
0
1
0
0
0
0
0
0
1
0
5
1
0
1
0
0
5
1
2
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
2
1
1
0
16
0
0
1
0
1
0
1
0
1
0
1
0
1
2
1
0
0
0
1
0
1
2
1
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
1
1
1
0
0
1
0
1
1
1
0
0
0
0
0
1
1
0
0
0
0
1
1
1
5
1
4
1
0
1
0
1
2
0
0
0
0
1
0
0
0
1
0
1
0
1
0
1
0
0
3
0
0
0
1
0
0
0
0
1
0
0
1
0
1
1
0
1
0
1
0
1
0
0
5
1
0
1
0
1
0
0
0
1
1
0
0
0
0
1
0
0
1
0
2
0
1
1
0
0
1
0
0
1
2
0
0
0
0
1
1
1
1
1
0
0
0
1
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
1
0
0
0
0
0
1
0
1
0

In [ ]:
def preprocess(reduce_train, reduce_test):
    for df in [reduce_train, reduce_test]:
        df['installation_session_count'] = df.groupby(['installation_id'])['Clip'].transform('count')
        df['installation_duration_mean'] = df.groupby(['installation_id'])['duration_mean'].transform('mean')
        #df['installation_duration_std'] = df.groupby(['installation_id'])['duration_mean'].transform('std')
        df['installation_title_nunique'] = df.groupby(['installation_id'])['session_title'].transform('nunique')
        
        df['sum_event_code_count'] = df[[2050, 4100, 4230, 5000, 4235, 2060, 4110, 5010, 2070, 2075, 2080, 2081, 2083, 3110, 4010, 3120, 3121, 4020, 4021, 
                                        4022, 4025, 4030, 4031, 3010, 4035, 4040, 3020, 3021, 4045, 2000, 4050, 2010, 2020, 4070, 2025, 2030, 4080, 2035, 
                                        2040, 4090, 4220, 4095]].sum(axis = 1)
        
        df['installation_event_code_count_mean'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('mean')
        #df['installation_event_code_count_std'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('std')
        
    features = reduce_train.loc[(reduce_train.sum(axis=1) != 0), (reduce_train.sum(axis=0) != 0)].columns # delete useless columns
    features = [x for x in features if x not in ['accuracy_group', 'installation_id']] + ['acc_' + title for title in assess_titles]
   
    return reduce_train, reduce_test, features
# call feature engineering function
reduce_train, reduce_test, features = preprocess(reduce_train, reduce_test)

In [ ]:
def run_feature_selection(reduce_train, reduce_test, usefull_features, new_features):
    kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(reduce_train), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature = categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature = categoricals)

        params = {
            'learning_rate': 0.01,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.7,
            'subsample': 0.7,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                          valid_sets=[train_set, val_set], verbose_eval = 500)
        oof_pred[val_ind] = model.predict(x_val)
    # using cohen_kappa because it's the evaluation metric of the competition
    loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    score = loss_score
    usefull_new_features = []
    for i in new_features:
        oof_pred = np.zeros((len(reduce_train), 4))
        evaluating_features = usefull_features + usefull_new_features + [i]
        print('Evaluating {} column'.format(i))
        print('Out best cohen kappa score is : {}'.format(score))
        for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
            print('Fold {}'.format(fold + 1))
            x_train, x_val = reduce_train[evaluating_features].iloc[tr_ind], reduce_train[evaluating_features].iloc[val_ind]
            y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = categoricals)
            val_set = lgb.Dataset(x_val, y_val, categorical_feature = categoricals)

            model = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                              valid_sets=[train_set, val_set], verbose_eval = 500)
            oof_pred[val_ind] = model.predict(x_val)
        loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
        print('Our new cohen kappa score is : {}'.format(loss_score))
        if loss_score > score:
            print('Feature {} is usefull, adding feature to usefull_new_features_list'.format(i))
            usefull_new_features.append(i)
            score = loss_score
        else:
            print('Feature {} is useless'.format(i))
        gc.collect()
    print('The best features are: ', usefull_new_features)
    print('Our best cohen kappa score is : ', score)

    return usefull_features + usefull_new_features

In [ ]:
def run_lgb(reduce_train, reduce_test, usefull_features):
    kf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(reduce_train), 4))
    y_pred = np.zeros((len(reduce_test), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)

        params = {
            'learning_rate': 0.07,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.7,
            'subsample': 0.7,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 1000000, early_stopping_rounds = 100, 
                          valid_sets=[train_set, val_set], verbose_eval = 100)
        oof_pred[val_ind] = model.predict(x_val)
        y_pred += model.predict(reduce_test[usefull_features]) / 10
    loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    result = pd.Series(np.argmax(oof_pred, axis = 1))
    print('Our oof cohen kappa score is: ', loss_score)
    print(result.value_counts(normalize = True))
    return y_pred

def predict(reduce_test, sample_submission, y_pred):
    sample_submission['accuracy_group'] = y_pred.argmax(axis = 1)
    sample_submission.to_csv('submission.csv', index = False)
    print(sample_submission['accuracy_group'].value_counts(normalize = True))

In [ ]:
features = reduce_train.loc[(reduce_train.sum(axis=1) != 0), (reduce_train.sum(axis=0) != 0)].columns # delete useless columns
features = [x for x in features if x not in ['accuracy_group', 'installation_id']]
reduce_train[features]

In [ ]:
y_pred = run_lgb(reduce_train, reduce_test, features)
predict(reduce_test, sample_submission, y_pred)

In [ ]:
params = {'n_estimators':2000,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.04,
            'feature_fraction': 0.9,
         'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100, 'eval_metric': 'cappa'
            }

In [ ]:
y = reduce_train['accuracy_group']
n_fold = 5
folds = GroupKFold(n_splits=n_fold)
cols_to_drop = ['game_session', 'installation_id', 'timestamp', 'accuracy_group', 'timestampDate']

In [ ]:
mt = MainTransformer()
ft = FeatureTransformer()
transformers = {'ft': ft}
regressor_model1 = RegressorModel(model_wrapper=LGBWrapper_regr())
regressor_model1.fit(X=reduce_train, y=y, folds=folds, params=params, preprocesser=mt, transformers=transformers,
                    eval_metric='cappa', cols_to_drop=cols_to_drop)

In [ ]:
from functools import partial
import scipy as sp
class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

In [ ]:
%%time
pr1 = regressor_model1.predict(reduce_train)

optR = OptimizedRounder()
optR.fit(pr1.reshape(-1,), y)
coefficients = optR.coefficients()

In [ ]:
opt_preds = optR.predict(pr1.reshape(-1, ), coefficients)
qwk(y, opt_preds)

In [ ]:
pr1 = regressor_model1.predict(reduce_test)
pr1[pr1 <= 1.12232214] = 0
pr1[np.where(np.logical_and(pr1 > 1.12232214, pr1 <= 1.73925866))] = 1
pr1[np.where(np.logical_and(pr1 > 1.73925866, pr1 <= 2.22506454))] = 2
pr1[pr1 > 2.22506454] = 3

In [ ]:
sample_submission['accuracy_group'] = pr1.astype(int)
sample_submission.to_csv('submission.csv', index=False)
sample_submission['accuracy_group'].value_counts(normalize=True)